In [1]:
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
CHAS_IDX=2
RM_IDX=4
PTRATIO=8

In [3]:
#Gather Data:
boston_dataset = load_boston()
data = pd.DataFrame(data=boston_dataset.data,columns=boston_dataset.feature_names)
features = data.drop(['INDUS','AGE'],axis=1)

log_prices=np.log(boston_dataset.target)
target = pd.DataFrame(data=log_prices,columns=['PRICE'])

In [4]:
property_stats=features.mean().values.reshape(1,11)

In [5]:
regr=LinearRegression()
regr.fit(features,target)
fitted_vals=regr.predict(features)

MSE = mean_squared_error(target,fitted_vals)
RMSE =  np.sqrt(MSE)

In [6]:
def get_log_estimate(nr_rooms,students_per_classroom,next_to_river=False,high_confidence=True):
    
    #Configure property:
    property_stats[0][RM_IDX]=nr_rooms
    property_stats[0][PTRATIO]=students_per_classroom
    
    if next_to_river:
        property_stats[0][CHAS_IDX]=1
    else:
        property_stats[0][CHAS_IDX]=0
    
    #Make prediction:
    log_estimate=regr.predict(property_stats)[0][0]
    
    #Calc Range:
    if high_confidence: # calculating 95% sd
        upper_bound=log_estimate+2*RMSE
        lower_bound=log_estimate-2*RMSE
        interval=95
    else: # Calculating 68% sd
        upper_bound=log_estimate-RMSE
        lower_bound=log_estimate-RMSE
        interval=68
    return log_estimate,upper_bound,lower_bound,interval

In [7]:
get_log_estimate(5,19,next_to_river=False)

(2.890381228132648, 3.2654054985269085, 2.5153569577383874, 95)

In [8]:
ZILLOW_MEDIAN_PRICE = 583.3 #Value from internet 
SCALE_FACTOR = ZILLOW_MEDIAN_PRICE/np.median(boston_dataset.target) # for converting into todays 
#price in $ with 1970's data

log_est,upper,lower,conf = get_log_estimate(9,15,next_to_river=False,high_confidence=False)

#Converting intpo today's dollar:
dollar_est = np.e**log_est * 1000 * SCALE_FACTOR
dollar_hi = np.e**upper * 1000 * SCALE_FACTOR
dollar_low = np.e**lower * 1000 * SCALE_FACTOR

#Rounding off the values:
rounded_est=np.around(dollar_est,-3)
rounded_hi=np.around(dollar_hi,-3)
rounded_low=np.around(dollar_low,-3)

#Printing the values:
print(f'The Estimated Property value in $:{rounded_est}')
print(f'Calculating at {conf} % standard deviation')
print(f'The Upper bound:{rounded_hi} ,The lower bound:{rounded_low}')

The Estimated Property value in $:827000.0
Calculating at 68 % standard deviation
The Upper bound:685000.0 ,The lower bound:685000.0


In [16]:
def get_dollar_estimate(rm,ptratio,chas=False,large_range=True):
    
    if rm<1 or ptratio<1:
        print('Invalid data. Try again')
        return
    
    log_est,upper,lower,conf = get_log_estimate(rm,ptratio,next_to_river=chas,high_confidence
                                                =large_range)

    #Converting intpo today's dollar:
    dollar_est = np.e**log_est * 1000 * SCALE_FACTOR
    dollar_hi = np.e**upper * 1000 * SCALE_FACTOR
    dollar_low = np.e**lower * 1000 * SCALE_FACTOR

    #Rounding off the values:
    rounded_est=np.around(dollar_est,-3)
    rounded_hi=np.around(dollar_hi,-3)
    rounded_low=np.around(dollar_low,-3)

    #Printing the values:
    print(f'The Estimated Property value is:{rounded_est}$')
    print(f'Calculating at {conf} % standard deviation')
    print(f'The Upper bound:{rounded_hi} ,The lower bound:{rounded_low}')

In [11]:
get_dollar_estimate(6,40)

The Estimated Property value in $:247000.0
Calculating at 95 % standard deviation
The Upper bound:360000.0 ,The lower bound:170000.0


In [19]:
#Run here:
print('Welcome to Boston House price prediction')
print('----------------------------------------')
print('Please enter the number of Rooms you prefer:')
Number_of_rooms_you_prefer = int(input())
print('Enter the Students to Teacher ratio which you prefer in the nearby school of your locality:')
Students_to_teacher_ratio = int(input())
print('\n')
print('RESULTS:')
get_dollar_estimate(Number_of_rooms_you_prefer,Students_to_teacher_ratio)

Welcome to Boston House price prediction
----------------------------------------
Please enter the number of Rooms you prefer:
5
Enter the Students to Teacher ratio which you prefer in the nearby school of your locality:
50


RESULTS:
The Estimated Property value is:155000.0$
Calculating at 95 % standard deviation
The Upper bound:226000.0 ,The lower bound:107000.0
